In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import prince
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from itertools import combinations

# Análise de Correspondência Múltipla (MCA)

In [ ]:
dados_mca = pd.read_csv("data/estudantes_adapta.csv")

# Fonte: adaptado de https://www.kaggle.com/datasets/mdmahmudulhasansuzan/students-adaptability-level-in-online-education
# Suzan et al.(2021) Students' Adaptability Level Prediction in Online Education using Machine Learning Approaches - DOI: 10.1109/ICCCNT51525.2021.9579741

In [3]:
# Tabelas de frequências das variáveis

print(dados_mca['Education'].value_counts(), "\n")
print(dados_mca['Institution'].value_counts(), "\n")
print(dados_mca['Financial'].value_counts(), "\n")
print(dados_mca['Internet'].value_counts(), "\n")
print(dados_mca['Adaptivity'].value_counts())

Education
School        530
University    456
College       219
Name: count, dtype: int64 

Institution
Non Government    823
Government        382
Name: count, dtype: int64 

Financial
Mid     878
Poor    242
Rich     85
Name: count, dtype: int64 

Internet
Mobile Data    695
Wifi           510
Name: count, dtype: int64 

Adaptivity
Moderate    625
Low         480
High        100
Name: count, dtype: int64


In [5]:
# Analisando as tabelas de contingência

# Gerando as tabelas de contingência em relação à "Adaptivity"

tabela_mca_1 = pd.crosstab(dados_mca["Adaptivity"], dados_mca["Education"])
tabela_mca_2 = pd.crosstab(dados_mca["Adaptivity"], dados_mca["Institution"])
tabela_mca_3 = pd.crosstab(dados_mca["Adaptivity"], dados_mca["Financial"])
tabela_mca_4 = pd.crosstab(dados_mca["Adaptivity"], dados_mca["Internet"])

print(tabela_mca_1, "\n")
print(tabela_mca_2, "\n")
print(tabela_mca_3, "\n")
print(tabela_mca_4, "\n")

Education   College  School  University
Adaptivity                             
High              3      47          50
Low             120     182         178
Moderate         96     301         228 

Institution  Government  Non Government
Adaptivity                             
High                 20              80
Low                 234             246
Moderate            128             497 

Financial   Mid  Poor  Rich
Adaptivity                 
High         36    22    42
Low         341   129    10
Moderate    501    91    33 

Internet    Mobile Data  Wifi
Adaptivity                   
High                 36    64
Low                 288   192
Moderate            371   254 



In [11]:
# Analisando a significância estatística das associações (teste qui²)

tab_4 = chi2_contingency(tabela_mca_4)

print("Adaptivity x Internet")
print(f"estatística qui²: {round(tab_4[0], 2)}")
print(f"p-valor da estatística: {round(tab_4[1], 4)}")
print(f"graus de liberdade: {tab_4[2]}")
print("-"*40)

tab_1 = chi2_contingency(tabela_mca_1)

print("Adaptivity x Education")
print(f"estatística qui²: {round(tab_1[0], 2)}")
print(f"p-valor da estatística: {round(tab_1[1], 4)}")
print(f"graus de liberdade: {tab_1[2]}")
print("-"*40)

tab_2 = chi2_contingency(tabela_mca_2)

print("Adaptivity x Institution")
print(f"estatística qui²: {round(tab_2[0], 2)}")
print(f"p-valor da estatística: {round(tab_2[1], 4)}")
print(f"graus de liberdade: {tab_2[2]}")
print("-"*40)

tab_3 = chi2_contingency(tabela_mca_3)

print("Adaptivity x Financial")
print(f"estatística qui²: {round(tab_3[0], 2)}")
print(f"p-valor da estatística: {round(tab_3[1], 4)}")
print(f"graus de liberdade: {tab_3[2]}")
print("-"*40)

Adaptivity x Internet
estatística qui²: 21.04
p-valor da estatística: 0.0
graus de liberdade: 2
----------------------------------------
Adaptivity x Education
estatística qui²: 38.69
p-valor da estatística: 0.0
graus de liberdade: 4
----------------------------------------
Adaptivity x Institution
estatística qui²: 107.11
p-valor da estatística: 0.0
graus de liberdade: 2
----------------------------------------
Adaptivity x Financial
estatística qui²: 236.86
p-valor da estatística: 0.0
graus de liberdade: 4
----------------------------------------


**Parametrização da MCA para três dimensões. O objetivo é criar um mapa perceptual 3D**

In [9]:
# Elaborando a MCA

mca = prince.MCA(n_components=3).fit(dados_mca)

## Quantidade total de dimensões

### Quantidade de dimensões = qtde total de categorias - qtde de variáveis

In [10]:
# Quantidade total de categorias
mca.J_

# Quantidade de variáveis na análise
mca.K_

# Quantidade de dimensões
quant_dim = mca.J_ - mca.K_

# Resumo das informações
print(f"quantidade total de categorias: {mca.J_}")
print(f"quantidade de variáveis: {mca.K_}")
print(f"quantidade de dimensões: {quant_dim}")

quantidade total de categorias: 13
quantidade de variáveis: 5
quantidade de dimensões: 8


In [12]:
# Obtendo os eigenvalues

tabela_autovalores = mca.eigenvalues_summary

print(tabela_autovalores)

          eigenvalue % of variance % of variance (cumulative)
component                                                    
0              0.321        20.06%                     20.06%
1              0.308        19.24%                     39.30%
2              0.258        16.10%                     55.40%


## Inércia principal total

In [13]:
# Soma de todos os autovalores (todas as dimensões existentes)

print(mca.total_inertia_)

1.5999999999999586


## Média da inércia principal total por dimensão

In [15]:
# É interessante plotar apenas dimensões com autovalores maiores do que a média

print(mca.total_inertia_/quant_dim)

0.19999999999999482


**Não é interessante trabalhar com dimensões onde o eigenvalue é menor do que a média dos eigenvalues**

*Neste caso, as 3 dimensões extraídas têm autovalores > 0.199*

In [16]:
# Obtendo as coordenadas principais das categorias das variáveis

coord_burt = mca.column_coordinates(dados_mca)

print(coord_burt)

                                   0         1         2
Education_College          -1.266116  0.559165 -0.011469
Education_School            0.476726 -0.675321  0.099547
Education_University        0.053979  0.516366 -0.110193
Institution_Government     -0.846907  0.380226  0.692520
Institution_Non Government  0.393097 -0.176484 -0.321437
Financial_Mid              -0.262792  0.125889 -0.393927
Financial_Poor              0.245959 -1.023834  0.979460
Financial_Rich              2.014228  1.614552  1.280454
Internet_Mobile Data       -0.057200 -0.631284  0.212550
Internet_Wifi               0.077949  0.860279 -0.289652
Adaptivity_High             1.972304  1.324840  1.302384
Adaptivity_Low             -0.676083 -0.030830  0.654504
Adaptivity_Moderate         0.203663 -0.188297 -0.711041


In [17]:
# Obtendo as coordenadas-padrão das categorias das variáveis

coord_padrao = mca.column_coordinates(dados_mca)/np.sqrt(mca.eigenvalues_)

print(coord_padrao)

                                   0         1         2
Education_College          -2.234816  1.007914 -0.022596
Education_School            0.841467 -1.217289  0.196122
Education_University        0.095278  0.930767 -0.217097
Institution_Government     -1.494873  0.685371  1.364369
Institution_Non Government  0.693853 -0.318119 -0.633279
Financial_Mid              -0.463853  0.226920 -0.776095
Financial_Poor              0.434140 -1.845496  1.929683
Financial_Rich              3.555306  2.910286  2.522688
Internet_Mobile Data       -0.100964 -1.137912  0.418756
Internet_Wifi               0.137588  1.550684 -0.570658
Adaptivity_High             3.481307  2.388070  2.565894
Adaptivity_Low             -1.193351 -0.055573  1.289472
Adaptivity_Moderate         0.359485 -0.339411 -1.400858


### Obtendo as coordenadas das observações do banco de dados

In [18]:
# Na função, as coordenadas das observações vêm das coordenadas-padrão

coord_obs = mca.row_coordinates(dados_mca)

print(coord_obs)

             0         1         2
0     0.164470  0.410168 -0.719597
1     0.116760 -0.127551 -0.521715
2    -0.739294  0.626296 -0.281168
3     0.265998 -0.557162 -0.439071
4     0.135029 -0.914878  0.640151
...        ...       ...       ...
1200 -0.612116  0.482365 -0.142631
1201 -0.301548  0.425598 -0.680697
1202  0.265998 -0.557162 -0.439071
1203 -0.612116  0.482365 -0.142631
1204  0.445596 -0.971645  0.102085

[1205 rows x 3 columns]


### Plotando o mapa perceptual (coordenadas-padrão)

In [19]:
# Primeiro passo: gerar um DataFrame detalhado

chart = coord_padrao.reset_index()

var_chart = pd.Series(chart['index'].str.split('_', expand=True).iloc[:,0])

nome_categ=[]
for col in dados_mca:
    nome_categ.append(dados_mca[col].sort_values(ascending=True).unique())
    categorias = pd.DataFrame(nome_categ).stack().reset_index()

chart_df_mca = pd.DataFrame({'categoria': chart['index'],
                             'obs_x': chart[0],
                             'obs_y': chart[1],
                             'obs_z': chart[2],
                             'variavel': var_chart,
                             'categoria_id': categorias[0]})

In [20]:
# Segundo passo: gerar o gráfico de pontos

fig = px.scatter_3d(chart_df_mca, 
                    x='obs_x', 
                    y='obs_y', 
                    z='obs_z',
                    color='variavel',
                    text=chart_df_mca.categoria_id)
fig.show()